In [4]:
from bs4 import BeautifulSoup, NavigableString, Tag
import requests
import lxml
import pandas as pd
import numpy as np
import re
import time
import os
import csv
import datetime as dt
pd.set_option('display.max_columns', 150)
pd.set_option('display.max_colwidth', -1)

### Read in list of HTX Beats

In [10]:
# https://cohgis-mycity.opendata.arcgis.com/datasets/fb3bb02ec56c4bb4b9d0cf3b8b3e5545_4?geometry=-97.322%2C29.405%2C-94.029%2C30.239
# dir = "/Users/Collier/Dropbox/Skills/Python/Projects/Real_Estate/HAR_scraping/"
dir = "/home/collier/Dropbox/Skills/Python/Projects/Real_Estate/HAR_scraping/"
beats = pd.read_csv(dir+"Houston_Police_Beats.csv")

#### Define list of Years and Months for search

In [11]:
years = [2010,2011,2012,2013,2014,2015,2016,2017]
months = ['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec']

#### Create Base DF to store results

In [12]:
dat_all_beats = pd.DataFrame(columns=['Date',
                       "Offense","Beat","# Offense",
                        "Blk Rng","Street"])

### Begin Scraping Iteration

In [15]:
len(beats)

116

In [17]:
# loop through beats list
HEADERS = {'User-Agent':'Mozilla/5.0'}
for idx,row in beats.iterrows():
    # loop through years
    dat_all_years = pd.DataFrame(columns=['Date',
                       "Offense",
                       "Beat",
                       "# Offense",
                        "Blk Rng",
                      "Street"])
    for year in years:
        yr = str(year)[-2:]
        dat_all_months = pd.DataFrame(columns=['Date',
                       "Offense",
                       "Beat",
                       "# Offense",
                        "Blk Rng",
                        "Street"])
        # loop through months
        for mon in months:
            BASE_URL = "http://www.houstontx.gov/police/cs/stats"+ \
                str(year)+"/"+str(mon)+str(yr)+"/"+str(mon)+str(yr)+ \
                str(row['Beats'].lower())+".htm"
            #begin scraping loop
            response = requests.get(BASE_URL,headers=HEADERS)
            soup = BeautifulSoup(response.content, "html5lib")
            table = soup.find("table", { "class" : "MsoNormalTable" })
            try:
                rows = table.find_all('tr')
            except:
                continue
            #append rows to beat level DF
            dat_all = pd.DataFrame(columns=['Date',
                       "Offense",
                       "Beat",
                       "# Offense",
                        "Blk Rng",
                      "Street"])
            for i in range(len(rows)):
                x = str(rows[i].text.replace("\n",""))
                words = x.split("      ") 

                if i > 0:
                    dat_stg = pd.DataFrame({'Date': [words[0]],
                       "Offense":[words[2]],
                       "Beat":[words[3]],
                       "# Offense":[words[9]],
                        "Blk Rng":[words[5]],
                      "Street":[words[6]]})
                    
                    dat_all = pd.concat([dat_all,dat_stg])
#                     dat_all['Month'] = mon
            dat_all_months = pd.concat([dat_all_months,dat_all])
#             dat_all_months['Year'] = year
            time.sleep(5)
        dat_all_years = pd.concat([dat_all_years,dat_all_months])
    dat_all_beats = pd.concat([dat_all_beats,dat_all_years])
    if idx % 10 == 0:
        print(float(idx/len(beats)))

IndexError: list index out of range

In [23]:
# row

In [35]:
words

['    6/28/2014', '15', 'Theft', '1A10', '\xa0', 'WOODRIDGE', '-', '-', '1   ']

### Testing/Dev

In [5]:
year = 2017
yr = 17
mon = "jan"

In [7]:
for idx,row in beats.iterrows():
    BASE_URL = "http://www.houstontx.gov/police/cs/beatpages/cs"+str(row['Beats'].lower())+".htm"

In [11]:
BASE_URL = "http://www.houstontx.gov/police/cs/stats"+str(year)+"/"+str(mon)+str(yr)+"/"+str(mon)+str(yr)+str(row['Beats'].lower())+".htm"
HEADERS = {'User-Agent':'Mozilla/5.0'}

In [12]:
BASE_URL

'http://www.houstontx.gov/police/cs/stats2017/jan17/jan1724c60.htm'

In [13]:
response = requests.get(BASE_URL,headers=HEADERS)
soup = BeautifulSoup(response.content, "html5lib")

In [14]:
table = soup.find("table", { "class" : "MsoNormalTable" })
rows = table.find_all('tr')

In [16]:
x = str(rows[0].text.replace("\n",""))

In [17]:
x

'    Date      Hour      Offense  Type      Beat      Premise      BlockRange      StreetName      Type      Suffix      #  offenses   '

In [18]:
words = x.split("      ") 

In [19]:
words

['    Date',
 'Hour',
 'Offense  Type',
 'Beat',
 'Premise',
 'BlockRange',
 'StreetName',
 'Type',
 'Suffix',
 '#  offenses   ']

In [60]:
idx_Date = [x for x, s in enumerate(words) if 'Date' in s]
idx_Offense = [x for x, s in enumerate(words) if 'Offense  Type' in s]
idx_Beat = [x for x, s in enumerate(words) if 'Beat' in s]
idx_num_offense = [x for x, s in enumerate(words) if '#  offenses' in s]
idx_blk_rng = [x for x, s in enumerate(words) if 'BlockRange' in s]
idx_street = [x for x, s in enumerate(words) if 'StreetName' in s]
# idx_street_type = [x for x, s in enumerate(words) if 'Type' in s]

In [61]:
idx_Date = str(idx_Date).replace("[","")
idx_Date = str(idx_Date).replace("]","")

idx_Offense = str(idx_Offense).replace("[","")
idx_Offense = str(idx_Offense).replace("]","")

idx_Beat = str(idx_Beat).replace("[","")
idx_Beat = str(idx_Beat).replace("]","")

idx_num_offense = str(idx_num_offense).replace("[","")
idx_num_offense = str(idx_num_offense).replace("]","")

idx_blk_rng = str(idx_blk_rng).replace("[","")
idx_blk_rng = str(idx_blk_rng).replace("]","")

idx_street = str(idx_street).replace("[","")
idx_street = str(idx_street).replace("]","")

In [62]:
dat_stg = pd.DataFrame({'Date': [words[int(idx_Date)]],
                       "Offense":[words[int(idx_Offense)]],
                       "Beat":[words[int(idx_Beat)]],
                       "# Offense":[words[int(idx_num_offense)]],
                        "Blk Rng":[words[int(idx_blk_rng)]],
                      "Street":[words[int(idx_street)]]})

In [63]:
dat_stg

,# Offense,Beat,Blk Rng,Date,Offense,Street
0,# offenses,Beat,BlockRange,Date,Offense Type,StreetName


In [ ]:


dat_stg = pd.DataFrame(words)
dat_stg = dat_stg.T
dat_stg.columns = ["Date","Hour","Offense",
                   "Beat","Premise","BlockRange",
                   "StreetName","Type","Suffix","#  offenses"]
dat_all = pd.concat([dat_all,dat_stg])